<a href="https://colab.research.google.com/github/nam4dev/chatbot_rasa_nlu_presentation/blob/master/chatbot_demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before executing the below cells, please execute following instructions:

*   In a terminal,
 * git clone https://github.com/nam4dev/chatbot_rasa_nlu_presentation.git
*   Copy/Paste (by drag & drop) following files in left panel (files/fichiers):
 * db_create_flights.sql
 * db_insert_flights.sql
 * training_dataset.json

In [0]:
# Setup VM with needed packages
!pip install rasa_nlu
!pip install sklearn_crfsuite

In [0]:
# Imports
import sqlite3

from rasa_nlu import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer

In [0]:
USER_QUESTIONS = (
    "I'd like to find a flight to Paris.",
    "I'm going to land in Mauritius and then I have to catch a flight to Taiwan.",
    "When I get there, I'll go up north for lunch, there's a great restaurant.",
    "Help me find a plane to India...",
)

In [0]:
# Model Classes
class Flight:
    """
    Class acting as a placeholder
    for a Flight row from the database
    """

    @property
    def price(self):
        """
        Property formatting the price with currency

        Returns:
            str: The formatted price
        """
        return f'{self._price} €'

    @property
    def company(self):
        """
        Property ensuring company is titled properly

        Returns:
            str: The titled company
        """
        return str(self._company).title()

    @property
    def destination(self):
        """
        Property ensuring destination is titled properly

        Returns:
            str: The titled destination
        """
        return str(self._destination).title()

    def __init__(self, row):
        """
        Constructor

        Args:
            row (list): A flight row
        """
        (self.pk,
         self._destination,
         self._price,
         self._company) = row

In [0]:
# ChatBot Class
class ChatBot:
    """
    Class representing a Chat bot
    """
    # Database file name
    DB_FILENAME = 'flights.db'
    # Training data set got from chatito
    TRAINING_DATA_FILE = './training_dataset.json'
    # SQL script for tables creation
    SQL_FLIGHTS_CREATE = './db_create_flights.sql'
    # SQL script for rows insertion
    SQL_FLIGHTS_INSERT = './db_insert_flights.sql'
    # Bot's potential responses
    RESPONSES = [
        "I'm sorry :( I couldn't find something interesting",
        "We've got a great flight to go to {f1.destinatio{f1.price}!",n} with {f1.company} for 
        '{f1.company} ({f1.price}) or {f2.company} ({f2.price}) would work to get to {f2.destination}!',
        '{f1.company} is one option for {f1.price}, but I know others too :)'
    ]
    # Rasa NLU Pipeline
    PIPELINE = [
        {"name": "SpacyNLP"},
        {"name": "SpacyTokenizer"},
        {"name": "CRFEntityExtractor"}
    ]

    def __init__(self):
        """
        Constructor
        """
        self.conn = None
        self.trainer = None
        self.interpreter = None
        self.trainer_config = None

    def initialize(self):
        """
        Template method to initialize:

            - Database
            - NLP Interpreter
        """
        self.initialize_db()
        self.initialize_interpreter()

    def initialize_db(self):
        """
        Initialize Database by:

            - Creating tables
            - Inserting sample data
        """
        self.conn = sqlite3.connect(ChatBot.DB_FILENAME)

        for script in (
                ChatBot.SQL_FLIGHTS_CREATE,  # Create database table
                ChatBot.SQL_FLIGHTS_INSERT  # Insert samples into the database
        ):
            with open(script) as fd:
                self.conn.executescript(fd.read())

    def initialize_trainer(self):
        """
        Initialize the NLP trainer by:

            - Creating Configuration instance
            - Creating Trainer instance with configuration
        """
        # Create a config that uses the pipeline
        self.trainer_config = RasaNLUModelConfig(
            configuration_values={"pipeline": ChatBot.PIPELINE}
        )
        # Create a trainer that uses the config
        self.trainer = Trainer(self.trainer_config)

    def initialize_interpreter(self):
        """
        Initialize the interpreter instance by:

            - Initializing the trainer
            - Training the interpreter
        """
        self.initialize_trainer()
        self.train_interpreter()

    def train_interpreter(self):
        """
        Get an interpreter instance by training the model
        (chatito training dataset)
        """
        # Create an interpreter by training the model
        self.interpreter = self.trainer.train(
            load_data(ChatBot.TRAINING_DATA_FILE)
        )

    def find_flights(self, params):
        """
        Find available flights based on given parameters

        Notes:
            If no parameters are provided, empty list is returned

        Args:
            params (dict): A map of SQL parameters

        Returns:
            list[Flight]: The list of available flight instances
        """
        if not params:
            return []
        # Create the base query
        query = 'SELECT * FROM flights'
        # Add filter clauses for each of the parameters
        if len(params) > 0:
            filters = [f'{k}=?' for k in params]
            query = f'{query} WHERE {" AND ".join(filters)}'
        # Create the tuple of values
        t = tuple(params.values())

        # Create a cursor
        c = self.conn.cursor()
        # Execute the query
        c.execute(query, t)
        # Return the results
        return [Flight(row) for row in c.fetchall()]

    def respond(self, message):
        """
        Respond to the given message (question)

        Args:
            message (str): The user message

        Returns:
            str: A formatted response
        """
        # Extract the entities
        entities = self.interpreter.parse(message)["entities"]
        # Initialize an empty params dictionary
        params = {}
        # Fill the dictionary with entities
        for ent in entities:
            params[ent["entity"]] = str(ent["value"])

        # Find flights that match the parameters
        flights = self.find_flights(params)
        # Get the names of the flights and index of the response
        n = min(len(flights), 3)
        # Build the formatted dict
        info = {f'f{i + 1}': flight for i, flight in enumerate(flights)}
        # Select the nth element of the responses list
        return ChatBot.RESPONSES[n].format(**info)

In [0]:
if __name__ == '__main__':
    bot = ChatBot()
    bot.initialize()

    for question in USER_QUESTIONS:
        print(f'[USER] {question}')
        print(f'[BOT] {bot.respond(question)}')

In [0]:
    #@title Interactive Questions
    question = "I'd like to go to California" #@param {type:"string"}

    print(f'[USER] {question}')
    print(f'[BOT] {bot.respond(question)}')